In [16]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
from moviepy.editor import VideoFileClip
from IPython.display import HTML
%matplotlib inline

def process_image(image):
    #grayscale the image
    gray = cv2.cvtColor(image,cv2.COLOR_RGB2GRAY)

    # Define a kernel size and apply Gaussian smoothing
    kernel_size = 5
    blur_gray = cv2.GaussianBlur(gray,(kernel_size, kernel_size),0)

    # Define our parameters for Canny and apply
    low_threshold = 50
    high_threshold = 150
    edges = cv2.Canny(blur_gray, low_threshold, high_threshold)

    # Next we'll create a masked edges image using cv2.fillPoly()
    mask = np.zeros_like(edges)   
    ignore_mask_color = 255   

    # This time we are defining a four sided polygon to mask
    imshape = image.shape
    vertices = np.array([[(0,imshape[0]),(350, 350), (550, 350), (imshape[1],imshape[0])]], dtype=np.int32)
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    masked_edges = cv2.bitwise_and(edges, mask)

    # Define the Hough transform parameters
    # Make a blank the same size as our image to draw on
    rho = 2 # distance resolution in pixels of the Hough grid
    theta = 1*np.pi/180 # angular resolution in radians of the Hough grid
    threshold = 15     # minimum number of votes (intersections in Hough grid cell)
    min_line_length = 10 #minimum number of pixels making up a line
    max_line_gap = 10    # maximum gap in pixels between connectable line segments
    line_image = np.copy(image)*0 # creating a blank to draw lines on

    # Run Hough on edge detected image
    # Output "lines" is an array containing endpoints of detected line segments
    lines = cv2.HoughLinesP(masked_edges, rho, theta, threshold, np.array([]),
                                min_line_length, max_line_gap)

    # Use polyfit to draw lines on a blank image
    left_x_arr = []
    left_y_arr = []
    right_x_arr = []
    right_y_arr = []
    for line in lines:
        for x1,y1,x2,y2 in line:
            slope = float((y2-y1)/(x2-x1))

            if not np.isnan(slope) or np.isinf(slope) or (slope == 0):
                #if (slope > -1) and (slope < -0.6):
                if (x1 + x2) < image.shape[1]:
                    left_x_arr.append(x1)
                    left_x_arr.append(x2)
                    left_y_arr.append(y1)
                    left_y_arr.append(y2)
                #if (slope > 0.2) and (slope < 1.5):
                if (x1 + x2) > image.shape[1]:
                    right_x_arr.append(x1)
                    right_x_arr.append(x2)
                    right_y_arr.append(y1)
                    right_y_arr.append(y2)
                    
    left_fit_para = np.polyfit(left_x_arr, left_y_arr, 1)
    right_fit_para = np.polyfit(right_x_arr, right_y_arr, 1)

    left_y_min = min(left_y_arr)
    right_y_min = min(right_y_arr)

    left_top_point = ((int)((left_y_min - left_fit_para[1])/left_fit_para[0]), left_y_min)
    left_bottom_point = ((int)((image.shape[0] - left_fit_para[1])/left_fit_para[0]), image.shape[0])
    cv2.line(line_image, left_top_point, left_bottom_point, (255, 0, 0), 20)

    right_top_point = ((int)((right_y_min - right_fit_para[1])/right_fit_para[0]), right_y_min)
    right_bottom_point = ((int)((image.shape[0] - right_fit_para[1])/right_fit_para[0]), image.shape[0])
    cv2.line(line_image, right_top_point, right_bottom_point, (255, 0, 0), 20)

    # Draw the lines on the edge image
    lines_edges = cv2.addWeighted(line_image, 1, image, 1, 0)
    return lines_edges

# Output videos
white_output = 'white.mp4'
clip1 = VideoFileClip("solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

white_output = 'yellow.mp4'
clip1 = VideoFileClip("solidYellowLeft.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

[MoviePy] >>>> Building video white.mp4
[MoviePy] Writing video white.mp4




  0%|          | 0/222 [00:00<?, ?it/s]

  5%|▍         | 10/222 [00:00<00:02, 98.67it/s]

  9%|▉         | 21/222 [00:00<00:02, 100.07it/s]

 14%|█▍        | 32/222 [00:00<00:01, 102.14it/s]

 19%|█▉        | 43/222 [00:00<00:01, 102.24it/s]

 23%|██▎       | 51/222 [00:00<00:01, 89.97it/s] 

 27%|██▋       | 59/222 [00:00<00:01, 86.59it/s]

 31%|███       | 68/222 [00:00<00:01, 85.29it/s]

 35%|███▍      | 77/222 [00:00<00:01, 85.06it/s]

 39%|███▊      | 86/222 [00:00<00:01, 85.30it/s]

 43%|████▎     | 95/222 [00:01<00:01, 84.18it/s]

 47%|████▋     | 104/222 [00:01<00:01, 84.54it/s]

 51%|█████     | 113/222 [00:01<00:01, 83.76it/s]

 55%|█████▍    | 122/222 [00:01<00:01, 82.67it/s]

 59%|█████▉    | 131/222 [00:01<00:01, 83.87it/s]

 63%|██████▎   | 140/222 [00:01<00:00, 84.23it/s]

 67%|██████▋   | 149/222 [00:01<00:00, 83.20it/s]

 71%|███████   | 158/222 [00:01<00:00, 84.31it/s]

 75%|███████▌  | 167/222 [00:01<00:00, 83.32it/s]

 79%|███████▉  | 176/222 [00:02<00:00, 81.94i

[MoviePy] Done.
[MoviePy] >>>> Video ready: white.mp4 

CPU times: user 2.28 s, sys: 691 ms, total: 2.98 s
Wall time: 2.96 s
[MoviePy] >>>> Building video yellow.mp4
[MoviePy] Writing video yellow.mp4




  0%|          | 0/682 [00:00<?, ?it/s]

  1%|▏         | 10/682 [00:00<00:06, 96.69it/s]

  3%|▎         | 21/682 [00:00<00:06, 98.86it/s]

  5%|▍         | 31/682 [00:00<00:06, 98.65it/s]

  6%|▌         | 42/682 [00:00<00:06, 99.27it/s]

  7%|▋         | 50/682 [00:00<00:06, 92.32it/s]

  9%|▊         | 59/682 [00:00<00:06, 89.41it/s]

 10%|▉         | 68/682 [00:00<00:07, 87.53it/s]

 11%|█▏        | 77/682 [00:00<00:07, 85.32it/s]

 13%|█▎        | 86/682 [00:00<00:06, 85.38it/s]

 14%|█▍        | 95/682 [00:01<00:07, 83.76it/s]

 15%|█▌        | 104/682 [00:01<00:06, 83.51it/s]

 17%|█▋        | 113/682 [00:01<00:06, 83.04it/s]

 18%|█▊        | 122/682 [00:01<00:06, 81.67it/s]

 19%|█▉        | 131/682 [00:01<00:06, 80.72it/s]

 21%|██        | 140/682 [00:01<00:06, 78.74it/s]

 22%|██▏       | 148/682 [00:01<00:06, 78.09it/s]

 23%|██▎       | 156/682 [00:01<00:06, 77.75it/s]

 24%|██▍       | 164/682 [00:01<00:06, 77.52it/s]

 25%|██▌       | 172/682 [00:02<00:06, 76.29it/s]

[MoviePy] Done.
[MoviePy] >>>> Video ready: yellow.mp4 

CPU times: user 7.85 s, sys: 2.18 s, total: 10 s
Wall time: 9.4 s
